# <center><strong>Important:</strong> Make a Copy of this Google Colab Notebook!
</center>

<p>Please refrain from using or modifying this current Google Colab notebook directly. Instead, follow these instructions to create your own copy:</p>

<ol>
  <li>Go to the "File" menu at the top of the Colab interface.</li>
  <li>Select "Save a copy in Drive" to create a duplicate of this notebook.</li>
  <li>You can now work on your own copy without affecting the original.</li>
</ol>

<p>This ensures that you have a personalized version to work on and make changes according to your needs. Remember to save your progress as you go. Enjoy working on your own copy of the Google Colab notebook!</p>

# **Module 21 Coding the Support Vector Machine Algorithm**
In this module, you will apply your knowledge from the past couple of modules to implement SVM using Python. The goal is to write the algorithm from scratch using Python for the task of binary classification. <p>

## **Getting Started**

Run the provided code sections and follow the instructions. Implement your own code were indicated.

##**Importing Python Packages**
The first step is to import your necessary Python packages.

In [ ]:
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn import datasets # Importing the datasets available from scikit-learn

## **Loading and Inspecting the Dataset**


First, we will load and inspect a dataset from scikit-learn. This dataset contains information about breast cancer tumors from digitized images of fine needle aspirates (FNA) of breast masses. This is a commonly used dataset for binary classification tasks, where the objective is to predict whether a tumor is malignant (cancerous) or benign (non-cancerous).

Within the dataset, there are 30 numerical features including the mean, standard error, and worst (largest) values of attributes such as radius, texture, perimeter, area, smoothness, compactness, concavity, concave points, symmetry, and fractal dimension.

The target variable represents the diagnosis of the tumor and is encoded as follows:
*   0: Malignant (indicating the presence of cancer)
*   1: Benign (indicating the absence of cancer)

In [ ]:
from pandas.io.common import dataclasses
cancer_data = datasets.load_breast_cancer()

data = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)

# Add the target variable to the DataFrame
data['diagnosis'] = cancer_data.target
diagnosis_map = {1:1, 0:-1} # Changing the values from 1:Malignant and 0:Benign to 1:Maligant and -1:Benign
data['diagnosis'] = data['diagnosis'].map(diagnosis_map)

First we will use `data.describe()` to inspect the features within the dataset and their distributions. These summary statistics show count, mean, standard deviation, minimum, maximum, and quartile information for numeric columns. This helps in understanding the central tendencies and distributions of the data. Inspecting these distributions is also useful for determining any inaccuracies within the dataset. For instance, in this dataset we have many metrics of size including the mean radius, mean perimeter, and mean area. We expect the minimum value for these features to be greater than zero since they are describing distances. By inspecting the data prior to running the algorithm, we can correct for any dataset errors.

In [ ]:
print(data.describe())

       mean radius  mean texture  mean perimeter    mean area  \
count   569.000000    569.000000      569.000000   569.000000   
mean     14.127292     19.289649       91.969033   654.889104   
std       3.524049      4.301036       24.298981   351.914129   
min       6.981000      9.710000       43.790000   143.500000   
25%      11.700000     16.170000       75.170000   420.300000   
50%      13.370000     18.840000       86.240000   551.100000   
75%      15.780000     21.800000      104.100000   782.700000   
max      28.110000     39.280000      188.500000  2501.000000   

       mean smoothness  mean compactness  mean concavity  mean concave points  \
count       569.000000        569.000000      569.000000           569.000000   
mean          0.096360          0.104341        0.088799             0.048919   
std           0.014064          0.052813        0.079720             0.038803   
min           0.052630          0.019380        0.000000             0.000000   
25%      

We can also inspect the first few rows of the dataset using `data.head(n)` where `n` is the number of rows you want to output. In this case, we will look at the first three rows of data. You can change the value for the number of rows to inspect more or less of the data.

In [ ]:
print(data.head(3))

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38           122.8     1001.0          0.11840   
1        20.57         17.77           132.9     1326.0          0.08474   
2        19.69         21.25           130.0     1203.0          0.10960   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0                 0.07871  ...          17.33            184.6      2019.0   
1                 0.05667  ...          23.41            158.8      1956.0   
2                 0.05999  ...          25.53            152.5      1709.0   

   worst smoothness  worst compactness  worst concavity  worst concave points  \


## **Feature Engineering and Splitting the Dataset**

Many times when you are working with data you'll be provided with raw data which may be incompatible with your model or hinder its performance. In this case, we'll need to do some pre-processing, or feature engineering, on the dataset before using our algorithm. For this dataset, we will do feature engineering through normalization. Normalization is the process of converting a range of values, into a standard range of values, typically in the interval [−1, 1] or [0, 1]. It’s not a strict requirement but it improves the speed of learning (e.g. faster convergence in gradient descent) and prevents numerical overflow.

In [ ]:
Y = data.loc[:, 'diagnosis']  # all rows of 'diagnosis', these are our targets
X = data.iloc[:, 1:30]  # all rows of column 1 and ahead, these are our features

# normalize the features using MinMaxScalar from sklearn.preprocessing
X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)

Next, we need to split the dataset into our training and testing datasets. This is necessary since we need to retain a testing dataset to see how the model will perform on unseen observations. To split the dataset, we'll use the function `train_test_split()` from sklearn.model_selection. The function takes in the following:
*   `X`: The feature data, typically represented as a NumPy array or pandas DataFrame.
*   `y`: The target variable or label data, corresponding to the feature data.
*   `test_size`: The proportion (between 0.0 and 1.0) of the dataset to include in the test split. For example, `test_size=0.2` would create a test set comprising 20% of the total data, while the remaining 80% is allocated to the training set.
*   `random_state`: The seed value used for random shuffling and splitting of the dataset. Setting a specific random_state ensures reproducibility of the split. If random_state is not provided, the data will be split differently each time the function is called.

The `train_test_split()` function returns four subsets:
*   `X_train`: The training set of feature data.
*   `X_test`: The test set of feature data.
*   `y_train`: The corresponding target variable for the training set.
*   `y_test`: The corresponding target variable for the test set.

In [ ]:
# first insert 1 in every row for intercept b
X.insert(loc=len(X.columns), column='intercept', value=1)

# test_size is the portion of data that will go into test set
# random_state is the seed used by the random number generator
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Now our dataset is ready to be used for binary classification with a support vector machine.

## **Cost Function**

The cost function, or objective function, is one of the main building blocks for our algorithm. This cost function is what we are trying to maximize or minimize depending on the machine learning algorithm. If you continue to learn more about machine learning, you'll encounter a wide variety of cost functions.

For SVM, our objective is to find a hyperplane that separates our samples with the highest possible margin while keeping misclassification as low as possible. We can achieve this by **minimizing** the following cost function:

$ J(\textbf{w}) = \frac{1}{2} \lVert \mathbf{w} \rVert^2 + C \bigg[ \frac{1}{N} ∑_{i}^{n} \text{max} (0,1-y_i * (\mathbf{w} \cdot x_i + b)) \bigg] $

The cost function can also be written as:

$ J(\textbf{w}) = \frac{1}{2} ⋅ λ \lVert \mathbf{w} \rVert ^2 + \frac{1}{N} ∑_{i}^{n} \text{max} (0,1-y_i * (\mathbf{w} \cdot x_i + b)) $

Using the cost function directly above, we can see that $\lambda$ is essentially equal to $1/C$ which means that a larger $\lambda$ will give a wider margin. Either of these cost functions can be used for SVM but it's important to remember what each regularization parameter (either $C$ or $\lambda$) does and then making adjustments to these paramters as necessary.

For our cost function, let's use the first function listed above. We'll start by computing the cost before moving on to its gradient which is what we'll be using for training.


In [ ]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = reg_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

From this cost function, you may notice that the intercept term $b$ is missing. In this example, the term has been included in the weight vector as follows:

$ f(x_i) = \tilde{w} \cdot \tilde{x_i} + w_0 = \textbf{w} \cdot x_i$

where $\textbf{w} = (\tilde{w},w_0), x_i = (\tilde{x}_i,1)$

This is why we added the extra column of 1s before splitting the dataset.

## **Gradient of Cost Function**

Now we need to compute the gradient of the cost function. Using our cost function, we can compute the gradient as:

$ \nabla J(\textbf{w}) = \frac{1}{N} \sum_i^n \begin{cases}
    \textbf{w} & \text{if max } (0,1-y_i * (\textbf{w} \cdot x_i)) = 0 \\
    \textbf{w} - C y_i x_i & \text{otherwise}
\end{cases} $

In [ ]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    Y_batch = np.array([Y_batch])
    X_batch = np.array([X_batch])

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

## **Training using Stochastic Gradient Descent (SGD)**

For the SVM algorithm, we need to minimize our cost function. This cost function is essentially a masure of how bad our model is performing. To find the minimum of $J(\textbf{w})$ we need to minimize $\lVert \mathbf{w} \rVert ^2$ which maximizes the margin ($2/\lVert \mathbf{w} \rVert$), and we need to minimize the sum of hinge loss which minimizes the misclassifications. The hinge loss function is: $ \text{max} (0,1-y_i * (w \cdot x_i + b))$. We minimize the cost function since both of our SVM objectives are achieved through minimization.

In order to minimize the cost function we wil be using Stochastic Gradient Descent (SGD). SGD is visually represented in the figure below.

<img src="https://sebastianraschka.com/images/faq/gradient-optimization/ball.png" alt="Image Description">

The gradient descent algorithm works as follows:
1.   Find the gradient of the cost function
2.   Move opposite to the gradient by a certain rate
3.   Repeat until convergence

Moving in the opposite direction to the gradient works because the gradient is the direction of fastest increase of the function. In order to reach the minimum of the function, we want to move in the opposite of that direction. This is why the fuction is called gradient **descent**. In the case of typical gradient descent, the first step is computed using all samples.

In our case, we want to use stochastic gradient descent which means that we are using a single example of the data at a time. This is beneficial since this method is efficient and converges quickly.



In [ ]:
def sgd(features, outputs):
    #max_epochs = 5000
    weights = np.zeros(features.shape[1])
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

    return weights

## **Training the Model**

Now we can train the model using our SGD function. To train the model, we need to also set values for the regularization strength and the learning rate. The regularization strength controls the trade-off between achieving a low training error and minimizing the complexity of the SVM model. The learning rate refers to a hyperparameter that controls the step size or rate at which the model parameters are updated during each iteration of the optimization process.

In [ ]:
reg_strength = 10000 # regularization strength
learning_rate = 0.000001 # step size
max_epochs = 5000 # number of times we do gradient descent

W = sgd(X_train.to_numpy(), y_train.to_numpy())
print("weights are: {}".format(W))

weights are: [-8.52781948e-01 -1.49560499e+00 -2.47602200e+00  1.24570577e+00
  3.24101058e+00 -3.25425106e+00 -6.89749508e+00  4.99782945e-01
 -2.03420387e-03 -5.68536285e+00  1.86156941e+00 -3.24168156e+00
 -3.79257070e+00 -1.65203504e+00  2.38378528e+00  1.74338092e+00
 -8.17056535e-01  1.97707839e+00  1.81484724e+00 -2.98566465e+00
 -5.29938880e+00 -1.27252924e+00 -3.27123761e+00 -2.16394532e+00
  6.11456534e-01 -2.60055045e+00 -3.92555214e-02 -4.69075257e+00
 -2.20470848e+00  9.15814564e+00]


## **Testing the Model**

Now we can test the model using the weights from training the model. Here we are using these weights to predict if the data for a given tumor is malignant or benign. We can compare the results to the actual known values for these tumors in order to output the accuracy, recall, and precision for the model.

*   **Precision**: Precision is a measure of the model's ability to correctly identify positive instances (true positives) out of all instances predicted as positive. It is calculated as the ratio of true positives (TP) to the sum of true positives and false positives (FP).
*   **Recall (Sensitivity or True Positive Rate)**: Recall measures the ability of the model to correctly identify positive instances (true positives) out of all actual positive instances. It is calculated as the ratio of true positives (TP) to the sum of true positives and false negatives (FN).
*   **Accuracy**: Accuracy measures the overall correctness of the model's predictions. It calculates the ratio of the number of correct predictions (true positives and true negatives) to the total number of instances. Accuracy provides an overall measure of the model's performance, considering both positive and negative predictions. However, accuracy alone may not be sufficient if the dataset is imbalanced (i.e., when the number of instances in one class is much higher than the other), as the model may achieve high accuracy by simply predicting the majority class.

In [ ]:
y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W))
    y_test_predicted = np.append(y_test_predicted, yp)

print("Accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
print("Recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
print("Precision on test dataset: {}".format(precision_score(y_test, y_test_predicted)))

accuracy on test dataset: 0.9736842105263158
recall on test dataset: 0.9859154929577465
precision on test dataset: 0.9722222222222222


## **Adjusting SVM Parameters**

Now let's make some adjustments to the parameters `reg_strength`, `learning_rate` and `max_epochs`. Change these values and note the changes in the accuracy, recall, and precision.

*   What happens when you increase the size of the learning rate?
*   What happens if you decrease the maximum number of epochs?
*   What happens when you decrease the regularization strength?
*   Adjust these three parameters to improve the accuracy, precision, and recall of the model compared to the original parameter values of:
```
reg_strength = 10000
learning_rate = 0.000001
max_epochs = 5000
```
Remember the original accuracy, recall, and precison were:
```
Accuracy on test dataset: 0.9736842105263158
Recall on test dataset: 0.9859154929577465
Precision on test dataset: 0.9722222222222222
```

✅ **Discuss the changes in accuracy, precision, and recall on Piazza or Discord with your fellow classmates!**










## ✨ **Congratulations you have now coded the SVM algorithm from scratch using Python!** ✨

This code is from https://towardsdatascience.com/svm-implementation-from-scratch-python-2db2fc52e5c2#7944 which provides additional information on coding the SVM algorithm from scratch.